In [ ]:
#Importing original dataset via CSV (https://archive.ics.uci.edu/dataset/468/online+shoppers+purchasing+intention+dataset)
import pandas as pd
def load_dataset(csv):
  df = pd.read_csv('online_shoppers_intention.csv')
  return df
csv = 'online_shoppers_intention.csv'
df = load_dataset(csv)

#Feature Engineering
#Converting Categorical into Integers and Use One-Hot Coding
sessions = df
sessions['Revenue'] = sessions['Revenue'].astype(int)#converting revenue into 0 and 1
sessions['SpecialDay'] = sessions['SpecialDay'].astype(int)
# One-hot encode VisitorType
visitor_type_dummies = pd.get_dummies(sessions['VisitorType'], prefix='VisitorType').astype(int)
sessions = pd.concat([sessions, visitor_type_dummies], axis=1)
sessions.drop('VisitorType', axis=1, inplace=True)
sessions['Weekend']=sessions['Weekend'].astype(int)
sessions['Month'] = sessions['Month'].map({'Feb':2, 'Mar':3,'May':5, 'June':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12})

#Adding Relevant Columns
sessions['SessionDuration']=sessions['Administrative_Duration']+sessions['Informational_Duration']+sessions['ProductRelated_Duration']
sessions['TotalPagesViewed']=sessions['Administrative']+sessions['Informational']+sessions['ProductRelated']
sessions['BounceRates'] = sessions['BounceRates'].astype(float)
sessions.head()

sessions.to_csv('updated_online_shoppers_intention.csv', sep=',', index=False)

In [ ]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTENC

def col_ins(ds, var):
    return [ds.columns.get_loc(col) for col in var]

def smotenc(X, y, cat_var_ins):
    sm = SMOTENC(random_state=42, categorical_features=cat_var_ins)
    return sm.fit_resample(X, y)

def df_smotenc(df, dep_var, cat_var):
    y = df[dep_var]
    X = df.drop(dep_var, axis=1)
    cat_var_ins = col_ins(X, cat_var)

    X_res, y_res = smotenc(X, y, cat_var_ins)

    X_res = pd.DataFrame(X_res, columns=X.columns)
    y_res = pd.DataFrame(y_res, columns=[dep_var])
    df_res = y_res.merge(X_res, left_index=True, right_index=True)

    return df_res

# Load your new dataset
new_dataset = pd.read_csv('new_dataset.csv')

# Apply SMOTENC
smote_output = df_smotenc(new_dataset, 'target', ('cat1', 'cat2', 'cat3'))

# Save the resampled dataset
smote_output.to_csv('smote_new_dataset.csv', sep=',', index=False)

In [ ]:
# -*- coding: utf-8 -*-
"""smotenc_revenue.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1UcJeKnQXi_1RIochdC930jvM2nGoH5XF
"""

import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTENC
import os

# Function to get column indices of categorical variables
def col_ins(ds, var):
  return [ds.columns.get_loc(col) for col in var]

# SMOTENC function for categorical and continuous data
def smotenc(X, y, cat_var_ins):
  sm = SMOTENC(random_state=42, categorical_features=cat_var_ins)
  return sm.fit_resample(X, y)

# Wrapper function for SMOTENC to handle DataFrame input and output
def df_smotenc(df, dep_var, cat_var):
  y = df[dep_var]
  X = df.drop(dep_var, axis=1)
  cat_var_ins = col_ins(X, cat_var)

  # Apply SMOTENC
  X_res, y_res = smotenc(X, y, cat_var_ins)

  # Convert back to DataFrame format
  X_res = pd.DataFrame(X_res, columns=X.columns)
  y_res = pd.DataFrame(y_res, columns=[dep_var])
  df_res = y_res.merge(X_res, left_index=True, right_index=True)

  return df_res

# List of categorical variables in your dataset
categorical_variables = [
    'Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 
    'Weekend', 'VisitorType_New_Visitor', 'VisitorType_Other', 'VisitorType_Returning_Visitor'
]

# List of file roots and folds for training data (update if necessary)
fileroots = ['all']
kfold = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

# Process each file and apply SMOTENC
for root in fileroots:
  for k in kfold:
    inputfile = ''.join([root, '_', k, '_', 'train.csv'])
    trainingset = pd.read_csv('updated_online_shoppers_intention.csv')
    
    # Apply SMOTENC with 'Revenue' as the target variable
    smote_output = df_smotenc(trainingset, 'Revenue', categorical_variables)
    
    # Save the output file in the desired directory
    outputfile = '_'.join(['SM', inputfile])
    outputfile = '/'.join(['SMOTE_files', outputfile])
    smote_output.to_csv(outputfile, sep=',', index=False)
